In [3]:
import rioxarray
import rasterio
import geopandas as gpd
import numpy as np
import pandas as pd
import pyproj
import json
from shapely.geometry import Polygon
import os
import sys
from tqdm import tqdm
import requests
import re

In [4]:
chip_dim_x = 224
chip_dim_y = 224
res = 30

In [23]:
states_name = ["ALABAMA", "ARIZONA", "ARKANSAS", "CALIFORNIA",
    "COLORADO", "CONNECTICUT", "DELAWARE",
    "FLORIDA", "GEORGIA", "IDAHO", "ILLINOIS",
    "INDIANA", "IOWA", "KANSAS", "KENTUCKY",
    "LOUISIANA", "MAINE", "MARYLAND", "MASSACHUSETTS",
    "MICHIGAN", "MINNESOTA", "MISSISSIPPI", "MISSOURI",
    "MONTANA", "NEBRASKA", "NEVADA", "NEW_HAMPSHIRE",
    "NEW_JERSEY", "NEW_MEXICO", "NEW_YORK", "NORTH_CAROLINA",
    "NORTH_DAKOTA", "OHIO", "OKLAHOMA", "OREGON",
    "PENNSYLVANIA", "RHODE_ISLAND", "SOUTH_CAROLINA",
    "SOUTH_DAKOTA", "TENNESSEE", "TEXAS", "UTAH",
    "VERMONT", "VIRGINIA", "WASHINGTON", "WEST_VIRGINIA",
    "WISCONSIN", "WYOMING"]

# for state_name in states_name:
#     state_name = state_name.lower()
#     print(state_name)
state_name = 'ALABAMA'
state_name = state_name.lower()
chips_df_to_sample = pd.read_csv(f"{state_name}/cdl_{state_name}.csv")

features = []
chip_height = chip_dim_y * res
chip_width = chip_dim_x * res
for row in chips_df_to_sample.iloc[:, :].iterrows():
    feature_template = [{'type': 'Feature',
                'properties': {'id': 0,
                              'center': [0, 0]},
                'geometry': {'type': 'MultiPolygon',
                'coordinates': [[[[-281415, 1533615],
                                  [256185, 1533615],
                                  [256185, 996015],
                                  [-281415, 996015],
                                  [-281415, 1533615]]]]
                            }
               }]
    # The subtraction/addition of `res` is needed in the following to change the x and y from center of the pixel to the edge
    chip_x0 = row[1]["chip_coordinate_x"] - res / 2
    chip_y0 = row[1]["chip_coordinate_y"] + res / 2

    coor = [[[[chip_x0, chip_y0],
              [chip_x0, chip_y0 - chip_height],
              [chip_x0 + chip_width, chip_y0 - chip_height],
              [chip_x0 + chip_width, chip_y0],
              [chip_x0, chip_y0]
         ]]]
    chip_cen = [chip_x0 + chip_width / 2, chip_y0 - chip_height / 2]
    chip_cen_lon, chip_cen_lat = [chip_cen[0], chip_cen[1]]
    
    feature_template[0]['geometry']['coordinates'] = coor
    feature_template[0]['properties']['id'] = row[1]["chip_id"]
    feature_template[0]['properties']['center'] = [chip_cen_lon, chip_cen_lat] 
    features.extend(feature_template)

feature_collection = {
'type': 'FeatureCollection',
 'name': 'chips',
 'crs': {'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:EPSG::5070'}},
 'features': features
}

with open(f"{state_name}/chip_bbox_task_{state_name}_5070.geojson", 'w') as outfile:
    json.dump(feature_collection, outfile)

# features = []
# proj = pyproj.Transformer.from_crs(5070, 4326, always_xy=True)
# chip_height = chip_dim_y * res
# chip_width = chip_dim_x * res
# for row in chips_df_to_sample.iloc[:5000, :].iterrows():
#     feature_template = [{'type': 'Feature',
#                 'properties': {'id': 0,
#                               'center': [0, 0]},
#                 'geometry': {'type': 'MultiPolygon',
#                 'coordinates': [[[[-281415, 1533615],
#                                   [256185, 1533615],
#                                   [256185, 996015],
#                                   [-281415, 996015],
#                                   [-281415, 1533615]]]]
#                             }
#                }]
#     # The subtraction/addition of `res` is needed in the following to change the x and y from center of the pixel to the edge
#     chip_x0 = row[1]["chip_coordinate_x"] - res / 2 
#     chip_y0 = row[1]["chip_coordinate_y"] + res / 2

#     coor = [[[[proj.transform(chip_x0, chip_y0)[0], proj.transform(chip_x0, chip_y0)[1]],
#               [proj.transform(chip_x0, chip_y0 - chip_height)[0], proj.transform(chip_x0, chip_y0 - chip_height)[1]],
#               [proj.transform(chip_x0 + chip_width, chip_y0 - chip_height)[0], proj.transform(chip_x0 + chip_width, chip_y0 - chip_height)[1]],
#               [proj.transform(chip_x0 + chip_width, chip_y0)[0], proj.transform(chip_x0 + chip_width, chip_y0)[1]],
#               [proj.transform(chip_x0, chip_y0)[0], proj.transform(chip_x0, chip_y0)[1]]
#          ]]]
#     chip_cen = [chip_x0 + chip_width / 2, chip_y0 - chip_height / 2]
#     chip_cen_lon, chip_cen_lat = proj.transform(chip_cen[0], chip_cen[1])
    
#     feature_template[0]['geometry']['coordinates'] = coor
#     feature_template[0]['properties']['id'] = row[1]["chip_id"]
#     feature_template[0]['properties']['center'] = [chip_cen_lon, chip_cen_lat] 
#     features.extend(feature_template)

# feature_collection = {
# 'type': 'FeatureCollection',
#  'name': 'chips',
#  'crs': {'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:EPSG::4326'}},
#  'features': features
# }

# with open(f"{state_name}/chip_bbox_task_{state_name}_4326.geojson", 'w') as outfile:
#     json.dump(feature_collection, outfile)
print('Sucessfully Completed')

Sucessfully Completed


In [24]:
features = []
proj = pyproj.Transformer.from_crs(5070, 4326, always_xy=True)
chip_height = chip_dim_y * res
chip_width = chip_dim_x * res
for row in chips_df_to_sample.iloc[:5000, :].iterrows():
    feature_template = [{'type': 'Feature',
                'properties': {'id': 0,
                              'center': [0, 0]},
                'geometry': {'type': 'MultiPolygon',
                'coordinates': [[[[-281415, 1533615],
                                  [256185, 1533615],
                                  [256185, 996015],
                                  [-281415, 996015],
                                  [-281415, 1533615]]]]
                            }
               }]
    # The subtraction/addition of `res` is needed in the following to change the x and y from center of the pixel to the edge
    chip_x0 = row[1]["chip_coordinate_x"] - res / 2 
    chip_y0 = row[1]["chip_coordinate_y"] + res / 2

    coor = [[[[proj.transform(chip_x0, chip_y0)[0], proj.transform(chip_x0, chip_y0)[1]],
              [proj.transform(chip_x0, chip_y0 - chip_height)[0], proj.transform(chip_x0, chip_y0 - chip_height)[1]],
              [proj.transform(chip_x0 + chip_width, chip_y0 - chip_height)[0], proj.transform(chip_x0 + chip_width, chip_y0 - chip_height)[1]],
              [proj.transform(chip_x0 + chip_width, chip_y0)[0], proj.transform(chip_x0 + chip_width, chip_y0)[1]],
              [proj.transform(chip_x0, chip_y0)[0], proj.transform(chip_x0, chip_y0)[1]]
         ]]]
    chip_cen = [chip_x0 + chip_width / 2, chip_y0 - chip_height / 2]
    chip_cen_lon, chip_cen_lat = proj.transform(chip_cen[0], chip_cen[1])
    
    feature_template[0]['geometry']['coordinates'] = coor
    feature_template[0]['properties']['id'] = row[1]["chip_id"]
    feature_template[0]['properties']['center'] = [chip_cen_lon, chip_cen_lat] 
    features.extend(feature_template)

feature_collection = {
'type': 'FeatureCollection',
 'name': 'chips',
 'crs': {'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:EPSG::4326'}},
 'features': features
}

with open(f"{state_name}/chip_bbox_task_{state_name}_4326.geojson", 'w') as outfile:
    json.dump(feature_collection, outfile)